In [1]:
import data_loader
import numpy as np

In [2]:
# df = data_loader.load_train_data("data/adult.data")
df = np.genfromtxt("data/adult.data",delimiter=",", dtype="|U", autostrip=True)
df.shape

(32561, 15)

In [3]:
(Xtrain, Ytrain, Xval, Yval) = data_loader.load_train_data("data/adult.data", is_df=False)

In [4]:
Xtrain.shape

(29315, 14)

In [5]:
Yval

array([0, 1, 1, ..., 0, 0, 1])

In [9]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

In [10]:
clf.fit(Xtrain, Ytrain)

ValueError: could not convert string to float: ' Local-gov'